In [29]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [30]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [31]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw pose connections


In [32]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [51]:
cap =  cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.3, min_tracking_confidence= 0.3) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        image, results = mediapipe_detection(frame, holistic)
        
        print(results)
        draw_landmarks(image, results)
        cv2.waitKey(15)
        cv2.imshow('Raw Webcam Feed', image)
        if cv2.waitKey(10) == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [52]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([ lh, rh])

In [53]:
test = extract_keypoints(results)
test.shape
print(test)

[ 5.08260429e-01  8.00191164e-01  2.36108789e-07  4.57932860e-01
  7.05306411e-01  1.01412134e-02  4.42276120e-01  6.23227477e-01
  5.83722116e-03  4.21370238e-01  5.67785621e-01 -6.32318342e-03
  4.00980026e-01  5.10091662e-01 -1.99579392e-02  5.20540774e-01
  5.35917342e-01  1.53913945e-02  5.29863298e-01  4.46718216e-01
 -2.41902471e-03  5.20379901e-01  3.89048100e-01 -2.04915721e-02
  5.03002524e-01  3.36702406e-01 -3.53394821e-02  5.35671771e-01
  5.46550035e-01 -9.99234430e-03  5.70453465e-01  4.48151886e-01
 -2.93112341e-02  5.65569878e-01  3.76430571e-01 -4.80310768e-02
  5.52405953e-01  3.16257268e-01 -6.26956671e-02  5.42680144e-01
  5.73338985e-01 -3.79038230e-02  5.86790025e-01  4.82316971e-01
 -5.66857234e-02  5.83328247e-01  4.13275808e-01 -7.16444775e-02
  5.72108090e-01  3.52701426e-01 -8.41133967e-02  5.42455792e-01
  6.12251699e-01 -6.59676045e-02  5.86952269e-01  5.56426466e-01
 -8.40908289e-02  5.93858004e-01  5.02765477e-01 -9.44206342e-02
  5.93161523e-01  4.49894

In [76]:
data_path = os.path.join('hand_data')
action = np.array(['help_sign_side_left', 'help_sign_side_right'])

# Thirty videos worth of data
no_sequences = 40

# Videos are going to be 30 frames in length
sequence_length = 40




In [90]:
for actions in action:
    for sequence in range(no_sequences):

        try:
            os.makedirs(os.path.join(data_path, actions , str(sequence)))
        except:
            pass

In [91]:
cap =  cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.3, min_tracking_confidence= 0.3) as holistic:
    
                
    for actions in action:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                ret, frame = cap.read()
                
                image, results = mediapipe_detection(frame, holistic)
                
                print(results)
                draw_landmarks(image, results)
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(actions, sequence), (15,12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0, 255), 1, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(actions, sequence), (15,12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('frame', image)
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(data_path, actions, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                
                if cv2.waitKey(5)  & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [ ]:
cap.release()
cv2.destroyAllWindows()